# 📊 Naked Put Selling Strategy: SPX, NDX & RUT Analysis

**Strategy:** Selling naked put options when indices trade below their 9-day moving average  
**Period:** January 2012 - October 2023  
**Starting Capital:** $250,000

This notebook analyzes the performance of a mean-reversion options strategy that profits from market bounces by selling puts during temporary weakness.

In [1]:
# Import required libraries
import json
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from datetime import datetime

# Load the strategy data
with open(r'c:\Users\Naeem\Downloads\Hipster Violet Zebra.json', 'r') as f:
    strategy_data = json.load(f)

print("✅ Data loaded successfully!")
print(f"Strategy Name: {strategy_data['state']['Name']}")
print(f"Period: {strategy_data['algorithmConfiguration']['startDate'][:10]} to {strategy_data['algorithmConfiguration']['endDate'][:10]}")

✅ Data loaded successfully!
Strategy Name: Hipster Violet Zebra
Period: 2012-01-01 to 2023-10-01


## 📊 Load and Parse Profit/Loss Data

Let's extract the daily P&L data and prepare it for visualization.

In [3]:
# Extract P&L data
pnl_data = strategy_data['profitLoss']

# Convert to DataFrame
pnl_df = pd.DataFrame(list(pnl_data.items()), columns=['Date', 'PnL'])
pnl_df['Date'] = pd.to_datetime(pnl_df['Date'], format='ISO8601')
pnl_df = pnl_df.sort_values('Date')

# Calculate cumulative P&L
pnl_df['Cumulative_PnL'] = pnl_df['PnL'].cumsum()

# Starting equity
starting_equity = float(strategy_data['statistics']['Start Equity'])
pnl_df['Portfolio_Value'] = starting_equity + pnl_df['Cumulative_PnL']

print(f"Total Trades: {len(pnl_df)}")
print(f"Starting Equity: ${starting_equity:,.2f}")
print(f"Ending Equity: ${pnl_df['Portfolio_Value'].iloc[-1]:,.2f}")
print(f"Total P&L: ${pnl_df['Cumulative_PnL'].iloc[-1]:,.2f}")

# Display first and last few trades
print("\n📈 First 5 Trades:")
print(pnl_df.head())
print("\n📉 Last 5 Trades:")
print(pnl_df.tail())

Total Trades: 1615
Starting Equity: $250,000.00
Ending Equity: $233,088.00
Total P&L: $-16,912.00

📈 First 5 Trades:
                       Date   PnL  Cumulative_PnL  Portfolio_Value
0 2012-01-07 05:00:00+00:00  16.0            16.0         250016.0
1 2012-01-14 05:00:00+00:00  14.0            30.0         250030.0
2 2012-01-21 05:00:00+00:00   8.0            38.0         250038.0
3 2012-01-28 05:00:00+00:00  13.0            51.0         250051.0
4 2012-02-04 05:00:00+00:00  11.0            62.0         250062.0

📉 Last 5 Trades:
                                 Date    PnL  Cumulative_PnL  Portfolio_Value
1610 2023-09-18 13:31:00.001000+00:00 -102.0        -16769.0         233231.0
1611        2023-09-20 04:00:00+00:00   39.0        -16730.0         233270.0
1612 2023-09-20 04:00:00.001000+00:00   19.0        -16711.0         233289.0
1613        2023-09-21 04:00:00+00:00   56.0        -16655.0         233345.0
1614        2023-09-21 13:31:00+00:00 -257.0        -16912.0         2330

## 📈 Cumulative P&L Visualization

This chart shows the equity curve over the entire backtest period. Watch for drawdown periods (valleys) and recovery phases.

In [4]:
# Create an interactive cumulative P&L chart with Plotly
fig = go.Figure()

# Add cumulative P&L line
fig.add_trace(go.Scatter(
    x=pnl_df['Date'],
    y=pnl_df['Portfolio_Value'],
    mode='lines',
    name='Portfolio Value',
    line=dict(color='royalblue', width=2),
    fill='tozeroy',
    fillcolor='rgba(65, 105, 225, 0.1)'
))

# Add horizontal line at starting equity
fig.add_hline(
    y=starting_equity, 
    line_dash="dash", 
    line_color="gray",
    annotation_text=f"Starting Equity: ${starting_equity:,.0f}",
    annotation_position="right"
)

# Update layout
fig.update_layout(
    title={
        'text': '💰 Portfolio Equity Curve (2012-2023)',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 20, 'color': '#333'}
    },
    xaxis_title='Date',
    yaxis_title='Portfolio Value ($)',
    hovermode='x unified',
    template='plotly_white',
    height=600,
    showlegend=True,
    yaxis=dict(tickformat='$,.0f')
)

fig.show()

# Calculate some key metrics
max_value = pnl_df['Portfolio_Value'].max()
min_value = pnl_df['Portfolio_Value'].min()
final_value = pnl_df['Portfolio_Value'].iloc[-1]

print(f"\n📊 Equity Curve Statistics:")
print(f"Peak Portfolio Value: ${max_value:,.2f}")
print(f"Trough Portfolio Value: ${min_value:,.2f}")
print(f"Final Portfolio Value: ${final_value:,.2f}")
print(f"Max Drawdown from Peak: ${max_value - min_value:,.2f}")


📊 Equity Curve Statistics:
Peak Portfolio Value: $256,464.00
Trough Portfolio Value: $231,220.00
Final Portfolio Value: $233,088.00
Max Drawdown from Peak: $25,244.00


## 📊 Strategy Performance Statistics

Let's examine the key performance metrics from the backtest.

In [5]:
# Extract and display key statistics
stats = strategy_data['statistics']

# Create a formatted statistics DataFrame
stats_dict = {
    'Metric': [
        'Total Orders',
        'Win Rate',
        'Loss Rate',
        'Average Win',
        'Average Loss',
        'Profit-Loss Ratio',
        'Net Profit',
        'Compounding Annual Return',
        'Sharpe Ratio',
        'Sortino Ratio',
        'Maximum Drawdown',
        'Total Fees',
        'Expectancy',
        'Start Equity',
        'End Equity'
    ],
    'Value': [
        stats['Total Orders'],
        stats['Win Rate'],
        stats['Loss Rate'],
        stats['Average Win'],
        stats['Average Loss'],
        stats['Profit-Loss Ratio'],
        stats['Net Profit'],
        stats['Compounding Annual Return'],
        stats['Sharpe Ratio'],
        stats['Sortino Ratio'],
        stats['Drawdown'],
        stats['Total Fees'],
        stats['Expectancy'],
        stats['Start Equity'],
        stats['End Equity']
    ]
}

stats_df = pd.DataFrame(stats_dict)

# Create a visualization of key metrics
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Win/Loss Distribution', 'Risk Metrics', 'Return Metrics', 'Trade Statistics'),
    specs=[[{"type": "pie"}, {"type": "bar"}],
           [{"type": "bar"}, {"type": "table"}]]
)

# 1. Win/Loss Distribution (Pie Chart)
win_rate = float(stats['Win Rate'].rstrip('%'))
loss_rate = float(stats['Loss Rate'].rstrip('%'))

fig.add_trace(
    go.Pie(
        labels=['Wins', 'Losses'],
        values=[win_rate, loss_rate],
        marker=dict(colors=['#2ecc71', '#e74c3c']),
        hole=0.4
    ),
    row=1, col=1
)

# 2. Risk Metrics (Bar Chart)
risk_metrics = ['Sharpe Ratio', 'Sortino Ratio', 'Drawdown']
risk_values = [
    float(stats['Sharpe Ratio']),
    float(stats['Sortino Ratio']),
    -float(stats['Drawdown'].rstrip('%'))
]

fig.add_trace(
    go.Bar(
        x=risk_metrics,
        y=risk_values,
        marker=dict(color=['#3498db', '#9b59b6', '#e74c3c']),
        text=[f"{v:.2f}" for v in risk_values],
        textposition='outside'
    ),
    row=1, col=2
)

# 3. Return Metrics (Bar Chart)
return_metrics = ['Annual Return', 'Net Profit']
return_values = [
    float(stats['Compounding Annual Return'].rstrip('%')),
    float(stats['Net Profit'].rstrip('%'))
]

fig.add_trace(
    go.Bar(
        x=return_metrics,
        y=return_values,
        marker=dict(color=['#f39c12', '#e67e22']),
        text=[f"{v:.2f}%" for v in return_values],
        textposition='outside'
    ),
    row=2, col=1
)

# 4. Key Statistics Table
table_data = [
    ['Total Orders', stats['Total Orders']],
    ['Win Rate', stats['Win Rate']],
    ['Avg Win', stats['Average Win']],
    ['Avg Loss', stats['Average Loss']],
    ['P/L Ratio', stats['Profit-Loss Ratio']],
    ['Total Fees', stats['Total Fees']]
]

fig.add_trace(
    go.Table(
        header=dict(
            values=['<b>Metric</b>', '<b>Value</b>'],
            fill_color='#3498db',
            font=dict(color='white', size=12),
            align='left'
        ),
        cells=dict(
            values=[[row[0] for row in table_data], [row[1] for row in table_data]],
            fill_color='lavender',
            align='left',
            font=dict(size=11)
        )
    ),
    row=2, col=2
)

# Update layout
fig.update_layout(
    title_text="📈 Strategy Performance Dashboard",
    showlegend=False,
    height=800
)

fig.show()

# Print detailed statistics
print("\n" + "="*60)
print("📊 DETAILED PERFORMANCE STATISTICS")
print("="*60)
for _, row in stats_df.iterrows():
    print(f"{row['Metric']:<30} {row['Value']}")
print("="*60)


📊 DETAILED PERFORMANCE STATISTICS
Total Orders                   2902
Win Rate                       68%
Loss Rate                      32%
Average Win                    0.03%
Average Loss                   -0.16%
Profit-Loss Ratio              0.18
Net Profit                     -6.994%
Compounding Annual Return      -0.615%
Sharpe Ratio                   -1.792
Sortino Ratio                  -0.767
Maximum Drawdown               9.800%
Total Fees                     $1615.00
Expectancy                     -0.194
Start Equity                   250000
End Equity                     232516


## 📉 Drawdown Analysis

Understanding drawdowns is crucial for risk management. Let's visualize the underwater equity curve.

In [6]:
# Calculate drawdown
pnl_df['Peak'] = pnl_df['Portfolio_Value'].cummax()
pnl_df['Drawdown'] = (pnl_df['Portfolio_Value'] - pnl_df['Peak']) / pnl_df['Peak'] * 100

# Create drawdown chart
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('Portfolio Value with Peak', 'Drawdown %'),
    vertical_spacing=0.12,
    row_heights=[0.6, 0.4]
)

# Portfolio value and peak
fig.add_trace(
    go.Scatter(
        x=pnl_df['Date'],
        y=pnl_df['Portfolio_Value'],
        mode='lines',
        name='Portfolio Value',
        line=dict(color='royalblue', width=2)
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=pnl_df['Date'],
        y=pnl_df['Peak'],
        mode='lines',
        name='Peak Value',
        line=dict(color='green', width=1, dash='dash')
    ),
    row=1, col=1
)

# Drawdown
fig.add_trace(
    go.Scatter(
        x=pnl_df['Date'],
        y=pnl_df['Drawdown'],
        mode='lines',
        name='Drawdown',
        fill='tozeroy',
        line=dict(color='red', width=1),
        fillcolor='rgba(255, 0, 0, 0.3)'
    ),
    row=2, col=1
)

fig.update_xaxes(title_text="Date", row=2, col=1)
fig.update_yaxes(title_text="Portfolio Value ($)", tickformat='$,.0f', row=1, col=1)
fig.update_yaxes(title_text="Drawdown (%)", ticksuffix='%', row=2, col=1)

fig.update_layout(
    title_text="📉 Drawdown Analysis",
    height=800,
    hovermode='x unified',
    template='plotly_white'
)

fig.show()

# Print drawdown statistics
max_dd = pnl_df['Drawdown'].min()
max_dd_date = pnl_df.loc[pnl_df['Drawdown'].idxmin(), 'Date']

print(f"\n📉 Drawdown Statistics:")
print(f"Maximum Drawdown: {max_dd:.2f}%")
print(f"Max Drawdown Date: {max_dd_date.strftime('%Y-%m-%d')}")
print(f"\nDrawdown Recovery: {stats['Drawdown Recovery']} days")


📉 Drawdown Statistics:
Maximum Drawdown: -9.84%
Max Drawdown Date: 2022-11-03

Drawdown Recovery: 1256 days


## 📊 Monthly and Annual Returns Heatmap

Let's analyze the seasonality and consistency of returns across different time periods.

In [7]:
# Calculate monthly returns
pnl_df['Year'] = pnl_df['Date'].dt.year
pnl_df['Month'] = pnl_df['Date'].dt.month

# Group by year and month to get monthly P&L
monthly_pnl = pnl_df.groupby(['Year', 'Month'])['PnL'].sum().reset_index()

# Calculate monthly returns
monthly_pnl['Monthly_Return'] = monthly_pnl['PnL'] / starting_equity * 100

# Pivot for heatmap
heatmap_data = monthly_pnl.pivot(index='Year', columns='Month', values='Monthly_Return')

# Create heatmap
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
               'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

fig = go.Figure(data=go.Heatmap(
    z=heatmap_data.values,
    x=month_names,
    y=heatmap_data.index,
    colorscale='RdYlGn',
    zmid=0,
    text=np.round(heatmap_data.values, 2),
    texttemplate='%{text}%',
    textfont={"size": 10},
    colorbar=dict(title="Return (%)")
))

fig.update_layout(
    title='📅 Monthly Returns Heatmap (%)',
    xaxis_title='Month',
    yaxis_title='Year',
    height=600,
    yaxis=dict(autorange='reversed')
)

fig.show()

# Calculate annual returns
annual_returns = pnl_df.groupby('Year')['PnL'].sum() / starting_equity * 100

print("\n📊 Annual Returns:")
print(annual_returns.to_string())
print(f"\nAverage Annual Return: {annual_returns.mean():.2f}%")
print(f"Best Year: {annual_returns.idxmax()} ({annual_returns.max():.2f}%)")
print(f"Worst Year: {annual_returns.idxmin()} ({annual_returns.min():.2f}%)")


📊 Annual Returns:
Year
2012   -0.0128
2013    0.2156
2014   -0.2804
2015    0.2996
2016   -0.0704
2017    0.5092
2018    0.3840
2019    1.2568
2020   -2.1784
2021    0.5516
2022   -6.0972
2023   -1.3424

Average Annual Return: -0.56%
Best Year: 2019 (1.26%)
Worst Year: 2022 (-6.10%)


## 🎯 Key Insights and Strategy Analysis

### Strategy Performance Summary

**Overall Results (2012-2023):**
- **Net Return:** -6.99% over ~11 years
- **Win Rate:** 68% (consistent premium collection)
- **Sharpe Ratio:** -1.79 (negative risk-adjusted return)
- **Maximum Drawdown:** 9.8%

### Why the Strategy Struggled

**1. Profit-Loss Asymmetry**
- Average Win: 0.03% (small wins from premium collection)
- Average Loss: -0.16% (larger losses when assigned)
- P/L Ratio: 0.18 (losses are ~5.5x larger than wins)

**2. Tail Risk Exposure**
- Naked puts have unlimited downside if markets crash
- Major drawdowns occurred during:
  - 2018 volatility spike
  - March 2020 COVID-19 crash
  - 2022 bear market

**3. Negative Expectancy**
- Expectancy of -0.194 means the strategy loses money on average per trade
- Despite 68% win rate, the occasional large losses outweigh frequent small gains

### When This Strategy Could Work Better

1. **Strict Risk Management:** Use stop losses or dynamic position sizing
2. **Volatility Filtering:** Only trade when VIX is elevated (higher premiums)
3. **Hedging:** Add portfolio hedges during high uncertainty periods
4. **Shorter Expiration:** Use weekly options for faster premium decay
5. **Credit Spreads:** Instead of naked puts, use put spreads to cap losses

### Conclusion

The **mean-reversion logic is sound**, but naked put selling requires exceptional risk management. The strategy successfully collected premiums 68% of the time, but the occasional large losses during market stress periods eroded overall profitability.

For retail traders, consider using **put spreads** (buying a lower strike put to limit losses) or implementing **stricter position sizing** rules to survive drawdown periods.

In [ ]:
# Naked Put Selling Strategy Analysis: "Hipster Violet Zebra"

## Strategy Overview
This notebook analyzes a **naked put selling strategy** that trades on **SPY** (S&P 500 ETF), **QQQ** (Nasdaq-100 ETF), and **IWM** (Russell 2000 ETF).

### Strategy Logic
The strategy sells **naked put options** on the indices when their respective prices are **below their 9-day moving average (9-MA)**. This is a contrarian mean-reversion approach that capitalizes on market oversold conditions.

### Why This Strategy May Work

**1. Mean Reversion Principle**
- When markets fall below their short-term moving average, there's a statistical tendency to revert back to the mean
- The 9-day MA is short enough to identify temporary pullbacks but not so short as to generate excessive false signals

**2. Premium Collection in Weakness**
- Selling puts when the market is below the 9-MA means implied volatility is typically elevated
- Higher volatility = higher option premiums = better income for put sellers

**3. Statistical Edge**
- Markets tend to move up more than they move down over time (equity risk premium)
- Selling puts below the moving average means you're getting paid to potentially buy dips at favorable prices

**4. Risk-Reward Mechanics**
- If the market bounces back (as mean reversion suggests), the puts expire worthless and you keep the premium
- If assigned, you acquire shares at a discount during a dip, with the potential to benefit from the eventual recovery

Let's analyze the actual performance of this strategy from 2012 to 2023...